# Let's prepare!
Notebook for preparing our data.
We'll apply 3 types of transformations to it:
1. Imputing its null values;
2. Standardizing it;
3. Window it into time windows.

The processes will be applied to each file individually.
After done, we can join the results from each file to train our models.

### Imputing null values
Para as variáveis numéricas, realizaremos:
- se inteiramente nula: preencher zero; (TODO: testar usar média do seu tipo de anomalia)
- se parcialmente nula: interpolação linear + bfill + ffill.


Para 'class' realizaremos:
- se inteiramente nula: preencher com o seu tipo de anomalia; (TODO: verificar validade desta suposição)
- se parcialmente nula: bfill + ffill.

In [4]:
import sys
sys.path.append("..")  # Allows imports from sibling directories
from constants import config, utils
from absl import logging
import raw_data_manager.raw_data_inspector as rdi

logging.set_verbosity(logging.DEBUG)

In [8]:
split_train_dir = "/home/ubuntu/lemi_3w/data/dataset_converted_v10101_split-20_source-all_class-all_well-all_train"
train_metadata = rdi.RawDataInspector(
    split_train_dir,
    config.DIR_PROJECT_CACHE / "train_metadata.parquet",
    True
)

train_metadata_table = train_metadata.get_metadata_table()
train_metadata_table

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
74203bb,NORMAL,REAL,1.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-24 03:00:00,491415,17885
9fbd6f9,NORMAL,REAL,2.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-09 06:00:00,520154,17933
28804c5,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-08 09:00:31,349162,17970
fa71d94,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-23 19:00:00,279737,17949
07d061e,NORMAL,REAL,1.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-02-01 07:01:14,475846,17927
...,...,...,...,...,...,...,...
1def3c2,SEVERE_SLUGGING,REAL,14.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-09-20 11:00:53,977958,17948
ea66cf6,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2315903,61999
34f032a,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2259539,61999


In [9]:
from data_exploration.metric_acquisition import MetricAcquisition

metric_aquisition = MetricAcquisition(train_metadata_table)
cache_file_name = config.CACHE_NAME_TRAIN_MEAN_STD_DEV

mean_and_std_metric_table = metric_aquisition.get_mean_and_std_metric(
    cache_file_name, True
)
mean_metric_list = mean_and_std_metric_table['mean_of_means']
std_metric_list = mean_and_std_metric_table['mean_of_stds']

mean_metric_list

DEBUG:absl:Mean of means was adquired
             P-PDG         P-TPT       T-TPT     P-MON-CKP   T-JUS-CKP  \
0     0.000000e+00  1.420389e+07  116.794133  6.122810e+06   69.415705   
1     0.000000e+00  8.715824e+06  117.581821  1.539264e+06   77.332955   
2     4.203851e+07  1.806945e+07  118.569073  2.492268e+06   79.545939   
3     4.485805e+07  2.114707e+07  116.569785  1.224582e+07   61.885166   
4     0.000000e+00  1.001934e+07  119.033867  1.575567e+06   84.148449   
...            ...           ...         ...           ...         ...   
1577  2.424625e+07  1.511253e+07   44.579236  2.051035e+06   26.684643   
1578  1.539201e+07  8.925425e+06  120.387588  4.042629e+06   94.437207   
1579  2.077121e+07  1.138257e+07  119.183729  4.102729e+06   88.286792   
1580  1.660494e+07  9.154420e+06  122.202564  4.044520e+06  101.257945   
1581  1.898863e+07  1.076657e+07  116.316665  4.090173e+06   78.424427   

        P-JUS-CKGL       QGL  
0     3.052964e+06  0.000000  
1     2.320

P-PDG         1.650749e+07
P-TPT         1.517584e+07
T-TPT         1.061237e+02
P-MON-CKP     4.729793e+06
T-JUS-CKP     7.843213e+01
P-JUS-CKGL    3.501117e+08
QGL           2.603752e-01
Name: mean_of_means, dtype: float64

In [4]:
from transformation_manager import TransformationManager

folder_name = "dataset_converted_v10101_split-20_source-all_class-all_well-all_train"
transformation_manager = TransformationManager(train_metadata_table, folder_name)

2023-09-09 02:25:50.115898: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 02:25:50.118246: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 02:25:50.174014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-09 02:25:50.174907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 02:25:51.228863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:

transformation_manager.apply_transformations_to_table(
    output_parent_dir=config.DIR_PROJECT_DATA,
    sample_interval_seconds=60,
    num_timesteps_for_window=20,
    avg_variable_mean=mean_metric_list,
    avg_variable_std_dev=std_metric_list,
)

DONE:   0%|          | 0/1582 [00:00<?, ?it/s]

DEBUG:absl:output_path /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train/0/WELL-00006_20170209200114.npz
DEBUG:absl:output_path /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train/0/WELL-00002_20170611100000.npz
DEBUG:absl:event_grandparent_output_dir /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train
DEBUG:absl:output_path /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train/0/WELL-00001_20170524030000.npz
DEBUG:absl:event_grandparent_output_dir /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train
DEBUG:absl:event_grandparent_output_dir /home/ubuntu/lemi_3w/data/transform-isdt-dataset_converted_v10101_split-20_source-all_class-all_well-all_train
DEBUG:absl:event_input_path.parent /home/ubuntu/l

In [6]:
import sys

sys.path.append("..")  # Allows imports from sibling directories

from constants import utils, config
ex_path = "dataset_converted_v10101_split-20_source-all_class-all_well-all_train/4/WELL-00001_20170316120203.parquet"
ex_event = utils.get_event(config.DIR_PROJECT_DATA / ex_path)
ex_event[0:10]

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,
2017-03-16 12:02:03,38265830.0,13654450.0,117.1953,6029680.0,68.64587,3283309.0,NaN,0.0,4
2017-03-16 12:02:04,38265830.0,13654520.0,117.1947,6030228.0,68.64333,3283309.0,NaN,0.0,4
2017-03-16 12:02:05,38265830.0,13654580.0,117.1942,6030777.0,68.64080,3283308.0,NaN,0.0,4
2017-03-16 12:02:06,38265830.0,13654650.0,117.1937,6031326.0,68.63827,3283308.0,NaN,0.0,4
2017-03-16 12:02:07,38265830.0,13654710.0,117.1932,6031875.0,68.63573,3283308.0,NaN,0.0,4
2017-03-16 12:02:08,38265830.0,13654780.0,117.1926,6032423.0,68.63321,3283307.0,NaN,0.0,4
2017-03-16 12:02:09,38265830.0,13654840.0,117.1921,6032972.0,68.63068,3283307.0,NaN,0.0,4
2017-03-16 12:02:10,38265830.0,13654900.0,117.1916,6033521.0,68.62814,3283307.0,NaN,0.0,4
2017-03-16 12:02:11,38265830.0,13654970.0,117.1911,6034070.0,68.62561,3283306.0,NaN,0.0,4


In [7]:
n = 5
ex_event.resample(f'{n}s').mean()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
timestamp,,,,,,,,,
2017-03-16 12:02:00,38265830.0,13654485.0,117.19500,6029954.0,68.644600,3283309.0,NaN,0.0,4.0
2017-03-16 12:02:05,38265830.0,13654712.0,117.19316,6031874.6,68.635738,3283307.6,NaN,0.0,4.0
2017-03-16 12:02:10,38265830.0,13655032.0,117.19056,6034618.4,68.623076,3283305.8,NaN,0.0,4.0
2017-03-16 12:02:15,38265830.0,13655356.0,117.18796,6037362.4,68.610420,3283303.8,NaN,0.0,4.0
2017-03-16 12:02:20,38265830.0,13655678.0,117.18536,6040106.2,68.597762,3283301.6,NaN,0.0,4.0
...,...,...,...,...,...,...,...,...,...
2017-03-16 13:59:40,38265830.0,13901756.0,117.04692,5908803.6,69.137942,3282013.4,NaN,0.0,4.0
2017-03-16 13:59:45,38265830.0,13900112.0,117.04800,5903815.2,69.133580,3282016.0,NaN,0.0,4.0
2017-03-16 13:59:50,38265830.0,13898470.0,117.04908,5898826.4,69.129226,3282018.4,NaN,0.0,4.0


In [8]:
#ex_event.shape[0]
ex_event[module_constants.event_num_attribs].columns

NameError: name 'module_constants' is not defined

In [ ]:
import numpy as np
from constants import module_constants
#np.hstack([ex_event[c] for c in ex_event.columns])




ex_event_stacked_num_attribs = np.hstack([np.array(ex_event[c]).reshape((ex_event.shape[0], 1)) for c in ex_event[module_constants.event_num_attribs].columns])
ex_event_class_attribs = np.array(ex_event[module_constants.event_class_attrib])

ex_event_stacked_num_attribs[0], ex_event_class_attribs[0]

(array([3.826583e+07, 1.365445e+07, 1.171953e+02, 6.029680e+06,
        6.864587e+01, 3.283309e+06, 0.000000e+00]),
 array([4]))

In [ ]:

def split_sequences(sequences_input, sequence_output, n_steps):
    """split a multivariate sequence into samples"""
    X, y = list(), list()
    for i in range(len(sequences_input)):
        # find the end of this pattern
        end_ix = i + n_steps

        # check if we are beyond the dataset
        if end_ix > len(sequences_input):
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences_input[i:end_ix], sequence_output[end_ix-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
n_steps=3
X, y = split_sequences(ex_event_stacked_num_attribs, ex_event_class_attribs, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(2):
    print(X[i], y[i])

(7076, 3, 7) (7076, 1)
[[3.826583e+07 1.365445e+07 1.171953e+02 6.029680e+06 6.864587e+01
  3.283309e+06 0.000000e+00]
 [3.826583e+07 1.365452e+07 1.171947e+02 6.030228e+06 6.864333e+01
  3.283309e+06 0.000000e+00]
 [3.826583e+07 1.365458e+07 1.171942e+02 6.030777e+06 6.864080e+01
  3.283308e+06 0.000000e+00]] [4]
[[3.826583e+07 1.365452e+07 1.171947e+02 6.030228e+06 6.864333e+01
  3.283309e+06 0.000000e+00]
 [3.826583e+07 1.365458e+07 1.171942e+02 6.030777e+06 6.864080e+01
  3.283308e+06 0.000000e+00]
 [3.826583e+07 1.365465e+07 1.171937e+02 6.031326e+06 6.863827e+01
  3.283308e+06 0.000000e+00]] [4]


In [ ]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
in_seq3 = np.array([25, 35, 45, 55, 65, 75, 85, 95, 95])
in_seq4 = np.array([35, 45, 55, 65, 75, 85, 95, 95, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))

demo_event = np.hstack((in_seq1, in_seq2, in_seq3, in_seq4))
demo_event, out_seq

(array([[10, 15, 25, 35],
        [20, 25, 35, 45],
        [30, 35, 45, 55],
        [40, 45, 55, 65],
        [50, 55, 65, 75],
        [60, 65, 75, 85],
        [70, 75, 85, 95],
        [80, 85, 95, 95],
        [90, 95, 95, 95]]),
 array([ 25,  45,  65,  85, 105, 125, 145, 165, 185]))

In [ ]:
n_steps=3
X, y = split_sequences(demo_event, out_seq, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 4) (7,)
[[10 15 25 35]
 [20 25 35 45]
 [30 35 45 55]] 65
[[20 25 35 45]
 [30 35 45 55]
 [40 45 55 65]] 85
[[30 35 45 55]
 [40 45 55 65]
 [50 55 65 75]] 105
[[40 45 55 65]
 [50 55 65 75]
 [60 65 75 85]] 125
[[50 55 65 75]
 [60 65 75 85]
 [70 75 85 95]] 145
[[60 65 75 85]
 [70 75 85 95]
 [80 85 95 95]] 165
[[70 75 85 95]
 [80 85 95 95]
 [90 95 95 95]] 185


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

n_features = X.shape[2]

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



In [ ]:
model.fit(X, y, epochs=200, verbose=0)

# Preparing data from well id=5

In [10]:
split_train_dir_well_id_5 = config.DIR_PROJECT_DATA / ""
train_metadata_well_id_5 = rdi.RawDataInspector(
    split_train_dir_well_id_5,
    config.DIR_PROJECT_CACHE / "train_metadata_well_id_5.parquet",
    True
)

train_metadata_well_id_5.get_metadata_table()

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
98bafe5,NORMAL,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-14 02:00:00,121535,10595
49ab511,NORMAL,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-17 02:00:00,174605,10747
7b40db6,NORMAL,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-16 20:00:00,153211,10747
2525f7b,NORMAL,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-16 02:00:00,208650,10649
9c18c98,NORMAL,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-30 18:00:00,141210,10666
...,...,...,...,...,...,...,...
403eb8c,FLOW_INSTABILITY,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-06-27 17:01:12,162551,7129
f589560,FLOW_INSTABILITY,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-06-25 06:01:03,183906,7138
a742152,FLOW_INSTABILITY,REAL,5,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-06-24 16:01:24,181359,7103


In [12]:
from transformation_manager import TransformationManager

folder_name = "dataset_converted_v10101_split-20_source-all_class-all_well-5_train"
transformation_manager_well5 = TransformationManager(
    train_metadata_well_id_5.get_metadata_table(), 
    output_folder_base_name=folder_name
)

DEBUG:absl:TransformationManager initialized with 95 items.
            Folder name is dataset_converted_v10101_split-20_source-all_class-all_well-5_train.


In [13]:
transformation_manager_well5.apply_transformations_to_table(
    output_parent_dir=config.DIR_PROJECT_DATA,
    sample_interval_seconds=60,
    num_timesteps_for_window=20,
    avg_variable_mean=mean_metric_list,
    avg_variable_std_dev=std_metric_list,
)

DONE:   0%|          | 0/95 [00:00<?, ?it/s]